In [1]:
using TensorKit
import TensorKit.×
include("./iPEPS_fSU2/iPEPS.jl")
include("./CTMRG_fSU2/CTMRG.jl")
@show pkgversion(TensorKit)

pkgversion(TensorKit) = v"0.12.2"


v"0.12.2"

In [7]:
t = Matrix(randn(3, 3))
@show t
u = zeros(3, 3)
for ii in 1:size(t, 1)
    u[ii, ii] = sqrt(inv(t[ii, ii]))
end
@show u

t = [-2.0522782627257543 0.5423177425508124 -1.1879373561101403; -0.013071025714311738 -2.4154028710968296 -0.7890654726191135; 0.7354378565655822 0.8607956793147556 0.9664920276221284]


DomainError: DomainError with -0.487263359049488642416037009752471931278705596923828125:
NaN result for non-NaN input.

In [2]:
# pspace = GradedSpace{fSU₂}((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
pspace = GradedSpace{fSU₂}((0 => 2), (1//2 => 1))
aspacel = GradedSpace{fSU₂}((0 => 1), (1//2 => 1))
aspacet = GradedSpace{fSU₂}((0 => 1), (1//2 => 2))
aspacer = GradedSpace{fSU₂}((0 => 2), (1//2 => 2))
aspaceb = GradedSpace{fSU₂}((0 => 1), (1//2 => 1), (1 => 1))
# aspacel = aspacer = aspacet = aspaceb = pspace
@show pspace
nothing

pspace = Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1)


In [13]:
a = id(pspace)
@tensor contractcheck=true b[] := a[x, in] * a[in, x]
@show scalar(b)

@show maximum(convert(Array, a))
@show minimum(convert(Array, a))
@show one(eltype(a))
@show one(mapreduce(eltype, promote_type, storagetype.(a)))
nothing

scalar(b) = -1.215487821516742e-15
maximum(convert(Array, a)) = 1.0
minimum(convert(Array, a)) = 0.0
one(eltype(a)) = 1.0


MethodError: MethodError: no method matching length(::TensorMap{GradedSpace{ProductSector{Tuple{SU2Irrep, FermionParity}}, TensorKit.SortedVectorDict{ProductSector{Tuple{SU2Irrep, FermionParity}}, Int64}}, 1, 1, ProductSector{Tuple{SU2Irrep, FermionParity}}, TensorKit.SortedVectorDict{ProductSector{Tuple{SU2Irrep, FermionParity}}, Matrix{Float64}}, FusionTree{ProductSector{Tuple{SU2Irrep, FermionParity}}, 1, 0, 0, Nothing}, FusionTree{ProductSector{Tuple{SU2Irrep, FermionParity}}, 1, 0, 0, Nothing}})

Closest candidates are:
  length(!Matched::Core.Compiler.InstructionStream)
   @ Base show.jl:2777
  length(!Matched::TensorKit.FibonacciIterator)
   @ TensorKit ~/.julia/packages/TensorKit/gpuAY/src/sectors/anyons.jl:86
  length(!Matched::Core.SimpleVector)
   @ Base essentials.jl:770
  ...


In [ ]:
t1 = 2 * id(pspace)
t2 = inv(sqrt(t1))
t3 = sqrt(inv(t1))
@assert t2 ≈ t3

In [ ]:
a = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
@show space(a)  # [l, t, p; r, b]
@show space(a') # [r, b; l, t, p]

Xl, vl = leftorth(a, ((1, 2, 5), (3, 4)))
@show space(Xl)
@show space(vl)
nothing

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [ ]:
A = id(pspace)
@show A
@tensor x = A[a, a]
@show x
nothing

In [ ]:
include("./simple_update_fSU2/Models.jl")
para = Dict{Symbol,Any}()
para[:t] = 1.0
para[:U] = 8
para[:τlis] = [0.01, 0.03, 0.05, 0.1, 0.2, 0.3]
para[:Dk] = 10  # Dkept in the simple udate

pspace = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspacelr = pspace
aspacetb = pspace
Lx = 2
Ly = 2
ipeps = iPEPSΓΛ(pspace, aspacelr, aspacetb, Lx, Ly; dtype=ComplexF64)

τlis = para[:τlis]
t = para[:t]
U = para[:U]
Dk = para[:Dk]
Nit = length(τlis)
hij = Hubbard_hij(t, U)
τ = τlis[1]
gateNN = exp(-τ * hij)

@show space(ipeps[1, 1].Γ)
@show space(ipeps[1, 1].l)
@show space(ipeps[1, 1].t)
@show space(ipeps[1, 1].r)
@show space(ipeps[1, 1].b)
nothing

In [ ]:
pspace = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspacel = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspacer = aspacel
aspacet = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspaceb = aspacet

Lx = 2
Ly = 2
Ms = Vector{TensorMap}(undef, Lx * Ly)
for ii in 1:Lx*Ly
    Ms[ii] = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
    Ms[ii] = Ms[ii] / norm(Ms[ii])
end
@assert Ms[1] != Ms[2]
ipeps = iPEPS(Ms, Lx, Ly)

envs = iPEPSenv(ipeps)
χ = 10
Nit = 2
nothing

In [ ]:
@show numind(envs[1, 1].transfer.l)
@show numind(envs[1, 2].transfer.l)
envs[1, 1].transfer.l = id(pspace)
@show numind(envs[1, 1].transfer.l)
@show numind(envs[1, 1].transfer.r)
@show numind(envs[1, 1].transfer.t)
@show numind(envs[1, 1].transfer.b)
@show numind(envs[1, 2].transfer.l)
@assert envs[1, 1].corner.lt != envs[1, 2].corner.lt